# Model selection

In this notebook, we will benchmark different configurations (vectorization and classification) to find the best one to our use case.

## Libraries

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer

## Data import

In [3]:
df = pd.read_csv('../mbti_data.csv')
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


## Data preprocessing

We apply these steps :
- lowercase
- stemming
- stopwords removal
- noise removal

In [4]:
stop_words = set(stopwords.words('english'))
porter_stemmer = PorterStemmer()

def preprocessing(text, stemming_on=False, stop_words=stop_words, porter_stemmer=porter_stemmer):
    text = text.lower()                                          # Lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)                  # Noise removal (regex to remove punctuations)
    text = text.strip()                                          # Noise removal (extra spaces)
    words = text.split()                                         # Split sentence into list of words
    words = [w for w in words if not w in stop_words]            # Stopwords removal
    if(stemming_on):
        words = [porter_stemmer.stem(word) for word in words]    # Replace the word by its stem
    text = " ".join(words)                                       # Convert list of words into a sentence
    return text

In [5]:
df['posts_preproc_no_stemming'] = df['posts'].apply(lambda row: preprocessing(row))

In [6]:
df['posts_preproc_full'] = df['posts'].apply(lambda row: preprocessing(row, stemming_on=True))

In [7]:
df.head()

,type,posts,posts_preproc_no_stemming,posts_preproc_full
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,http www youtube com watch v qsxhcwe3krw http ...,http www youtub com watch v qsxhcwe3krw http 4...
1,ENTP,'I'm finding the lack of me in these posts ver...,finding lack posts alarming sex boring positio...,find lack post alarm sex bore posit often exam...
2,INTP,'Good one _____ https://www.youtube.com/wat...,good one https www youtube com watch v fhigbol...,good one http www youtub com watch v fhigbolff...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",dear intp enjoyed conversation day esoteric ga...,dear intp enjoy convers day esoter gab natur u...
4,ENTJ,'You're fired.|||That's another silly misconce...,fired another silly misconception approaching ...,fire anoth silli misconcept approach logic go ...


Below, an example of the posts after preprocessing's step (one without stemming and one with).

In [8]:
df.loc[1, 'posts_preproc_no_stemming'][:1000]

'finding lack posts alarming sex boring position often example girlfriend currently environment creatively use cowgirl missionary enough giving new meaning game theory hello entp grin takes converse flirting acknowledge presence return words smooth wordplay cheeky grins lack balance hand eye coordination real iq test score 127 internet iq tests funny score 140s higher like former responses thread mention believe iq test banish know entp vanish site year half return find people still commenting posts liking ideas thoughts know entp http img188 imageshack us img188 6422 6020d1f9da6944a6b71bbe6 jpg http img adultdvdtalk com 813a0c6243814cab84c51 think things sometimes go old sherlock holmes quote perhaps man special knowledge special powers like rather encourages seek complex cheshirewolf tumblr com 400 000 post really never thought e j p real functions judge use use ne ti dominates fe emotions rarely si also use ni due strength know though ingenious saying really want try see happens pla

In [9]:
df.loc[1, 'posts_preproc_full'][:1000]

'find lack post alarm sex bore posit often exampl girlfriend current environ creativ use cowgirl missionari enough give new mean game theori hello entp grin take convers flirt acknowledg presenc return word smooth wordplay cheeki grin lack balanc hand eye coordin real iq test score 127 internet iq test funni score 140 higher like former respons thread mention believ iq test banish know entp vanish site year half return find peopl still comment post like idea thought know entp http img188 imageshack us img188 6422 6020d1f9da6944a6b71bbe6 jpg http img adultdvdtalk com 813a0c6243814cab84c51 think thing sometim go old sherlock holm quot perhap man special knowledg special power like rather encourag seek complex cheshirewolf tumblr com 400 000 post realli never thought e j p real function judg use use ne ti domin fe emot rare si also use ni due strength know though ingeni say realli want tri see happen play first person shooter back drive around want see look rock paper one best make lol gu

## Vectorization

We benchmark 2 methods:
- CountVectorizer
- TfidfVectorizer

In [10]:
# CountVectorizer
corpus = df['posts_preproc_full']

In [11]:
type(corpus)

pandas.core.series.Series

In [12]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [13]:
type(X)

scipy.sparse.csr.csr_matrix

In [14]:
df.head()

,type,posts,posts_preproc_no_stemming,posts_preproc_full
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,http www youtube com watch v qsxhcwe3krw http ...,http www youtub com watch v qsxhcwe3krw http 4...
1,ENTP,'I'm finding the lack of me in these posts ver...,finding lack posts alarming sex boring positio...,find lack post alarm sex bore posit often exam...
2,INTP,'Good one _____ https://www.youtube.com/wat...,good one https www youtube com watch v fhigbol...,good one http www youtub com watch v fhigbolff...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",dear intp enjoyed conversation day esoteric ga...,dear intp enjoy convers day esoter gab natur u...
4,ENTJ,'You're fired.|||That's another silly misconce...,fired another silly misconception approaching ...,fire anoth silli misconcept approach logic go ...


## Prediction